# TO DO

- Check for correlations between variables and delete whatever seems to be unimportant.
- Make subsets of data and evaluate the results only for the subsets.

Notebook for the descriptive analysis of the databases used for the population and job synthesis. 

Initially there are 4 databases to analyze: From statistics Denmark we have a commuter matrix, a job (day) and a population (night) databases. From the DTU database, we have the sessions database.

In [2]:
### Basic module import
import os           # Working directory
import pandas as pd # Data processing
import numpy as np  # Scientific computing/matrix algebra
import matplotlib
import matplotlib.pyplot as plt # Common graphing interface (check also plotly and plotnine)
import re # Regular expressions

In [3]:
### Changing the working directory
#os.chdir('C:/Users/shgm/Desktop/projects/job_VAE') # PC
#os.chdir('/mnt/sdb1/data_shgm/') # Server 32
os.chdir('/mnt/sdb1/data_sergio/') # Server ai
print(os.getcwd())

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/sdb1/data_sergio/'

## Job data

In [ ]:
### Reading the job data
job_df = pd.read_csv('Job.csv', sep=',')
job_df.head()

In [ ]:
# How are the data types defined on the pandas loading
job_df.dtypes

In [ ]:
# Defining the 'categorical' and 'numerical' lists and changing the datatypes accordingly
categorical = ['Municipality', 'AgeGroup', 'Gender', 'Sector', 'Socio']
numerical   = ['Val', 'Year']

job_df[categorical] = job_df[categorical].astype('category')

In [ ]:
# Categorical data description
job_df[categorical].describe()

In [ ]:
# Numerical data description
job_df[numerical].describe()

In [ ]:
# Saving this set for further investigation
job_mun = set(job_df['Municipality'])

## Population data

In [3]:
# Read the population data
pop_df = pd.read_csv('pop.csv', sep=',')
pop_df.head()

,Municipality,edu,PopSocio,Sector,AgeGroup,Gender,Year,Val
0,101,H10,0,A,15-19,M,2009,2
1,101,H10,0,A,15-19,M,2010,1
2,101,H10,0,A,15-19,M,2011,2
3,101,H10,0,A,15-19,M,2012,1
4,101,H10,0,A,15-19,M,2013,3


In [ ]:
# How are the data types defined on the pandas loading
pop_df.dtypes

In [4]:
# Defining the 'categorical' and 'numerical' lists and changing the datatypes accordingly
categorical = ['Municipality', 'edu', 'PopSocio', 'Sector', 'AgeGroup', 'Gender']

pop_df[categorical] = pop_df[categorical].astype('category')

In [ ]:
# Categorical data description
pop_df[categorical].describe()

In [ ]:
# Numerical data description
pop_df[numerical].describe()

In [ ]:
pop_mun = set(pop_df['Municipality'])
# The 950 municipality appears on the occupation database but not on the population
pop_mun ^ job_mun

## Commuter matrix data

In [5]:
# Reading of the commuting matrix
#del job_mun, pop_mun

cm_df = pd.read_csv('commuter_codes.csv', sep=',')
cm_df.head()

,Year,Gender,Residence,Work,value
0,2008,K,1,1,133321
1,2008,K,1,2,31311
2,2008,K,1,3,6802
3,2008,K,1,4,69
4,2008,K,1,5,2770


In [ ]:
# How are the data types defined on the pandas loading
cm_df.dtypes

In [6]:
# Defining the 'categorical' and 'numerical' lists and changing the datatypes accordingly
categorical = ['Gender', 'Residence', 'Work']
numerical = ['Year', 'value']

cm_df[categorical] = cm_df[categorical].astype('category')

In [ ]:
# Categorical data description
cm_df[categorical].describe()

In [ ]:
# Numerical data description
cm_df[numerical].describe()

In [ ]:
# The 950 municipality appears on the work vector but not on the residence one
set(cm_df.Residence) ^ set(cm_df.Work)

In [7]:
# Commuter matrix with names instead of numbers
cm_df_val = pd.read_csv('commuter_values.csv', sep=',', encoding = "ISO-8859-1")
cm_df_val.head()

,Year,Gender,Residence,Work,value
0,2008,Kvinder,Landsdel Byen København,Landsdel Byen København,133321
1,2008,Kvinder,Landsdel Byen København,Landsdel Københavns omegn,31311
2,2008,Kvinder,Landsdel Byen København,Landsdel Nordsjælland,6802
3,2008,Kvinder,Landsdel Byen København,Landsdel Bornholm,69
4,2008,Kvinder,Landsdel Byen København,Landsdel Østsjælland,2770


In [ ]:
# The 950 municipality corresponds to "Uden for Danmark"
set(cm_df_val.Residence) ^ set(cm_df_val.Work)

In [ ]:
# Checking whether they are organized in the same way or not
all(cm_df.value == cm_df_val.value)

In [8]:
# Create a serial number to merge
cm_df['id'] = cm_df.index
cm_df_val['id'] = cm_df_val.index

# Merge the databases
cm_df_tot = cm_df.merge(cm_df_val, on='id', suffixes=('_c', '_v'))

# Convert the residence code for further work
cm_df_tot['Residence_c'] = cm_df_tot['Residence_c'].astype('int64')

In [ ]:
cm_df_tot.head()

In [ ]:
cm_df_tot.dtypes

In [9]:
names_codes = cm_df_tot[['Work_c', 'Work_v']].drop_duplicates()
names_codes.head()

,Work_c,Work_v
0,1,Landsdel Byen København
1,2,Landsdel Københavns omegn
2,3,Landsdel Nordsjælland
3,4,Landsdel Bornholm
4,5,Landsdel Østsjælland


In [ ]:
names_codes.tail()

## TU data

In [4]:
# Deletion of the commuter matrix data and reading of the sessions dataset
#del cm_df_val, cm_df_tot

ses_df = pd.read_csv('TU0616_csv/session.csv', sep=',') # 140672 obs
# ses_df = pd.read_csv('session.txt', sep=';', encoding = "ISO-8859-1")

FileNotFoundError: [Errno 2] File b'TU0616_csv/session.csv' does not exist: b'TU0616_csv/session.csv'

In [11]:
ses_df.head()

,SessionId,InterviewType,DiaryDate,DiaryYear,PseudoYear,DiaryMonth,DiaryWeekday,DiaryDaytype,HomeAdrNUTS,HomeAdrMunCode,...,DayNumJourneys,JstartType,JstartMuncode,JstartNTMzone,JstartNearestStation,JstartDistNearestStation,DayJourneyType,DayPrimTargetMuncode,DayPrimTargetPurp,SessionWeight
0,50068,0,13281,2006,2006/7,5,6,23,DK013,230,...,0.0,3.0,230.0,230045.0,Nærum ...,3.4,1,230.0,1.0,204.182196
1,50070,0,13282,2006,2006/7,5,7,32,DK012,159,...,0.0,3.0,159.0,159074.0,Stengården ...,0.4,1,159.0,1.0,495.028999
2,50071,0,13282,2006,2006/7,5,7,32,DK011,101,...,1.0,2.0,101.0,102151.0,Nørreport ...,0.4,11,101.0,31.0,460.598736
3,50072,0,13282,2006,2006/7,5,7,32,DK050,840,...,1.0,2.0,840.0,840052.0,Hobro ...,8.3,11,661.0,41.0,113.629100
4,50073,0,13282,2006,2006/7,5,7,32,DK012,183,...,0.5,2.0,183.0,183013.0,Ishøj ...,0.9,21,360.0,44.0,400.380203


In [12]:
pd.set_option('display.max_columns',103)
ses_df.describe()

,SessionId,InterviewType,DiaryDate,DiaryYear,DiaryMonth,DiaryWeekday,DiaryDaytype,HomeAdrMunCode,HomeAdrOldMuncode,HomeAdrCityCode,HomeAdrCitySize,HomeAdrNTMzone,HomeAdrDistNearestStation,HomeParkPoss,RespSex,RespYearBorn,RespAgeSimple,RespAgeCorrect,RespMainOccup,RespEdulevel,PrimOccMuncode,PrimOccOldMuncode,PrimOccNTMzone,WorkHoursPw,WorkHourType,WorkPubPriv,WorkatHomeDayspM,SduMuncode,SduOldMuncode,SduNTMzone,GISdistHW,kmarbud,HwDayspW,HwDaysReason,WorkParkPoss,RespHasBicycle,RespHasSeasonticket,RespHasRejsekort,ResphasDrivlic,RespDrivlicYear,RespIsmemCarshare,HousehNumcars,HousehCarOwnership,Handicap,HousehAccomodation,HousehAccOwnorRent,IncRespondent,IncRespondent2000,IncSpouse,IncSpouse2000,IncNuclFamily,IncNuclFamily2000,IncFamily,IncFamily2000,IncHouseh,IncHouseh2000,NuclFamType,PosInFamily,NuclFamNumPers,NuclFamNumAdults,NuclFamNumPers1084,NuclFamNumDrivLic,FamNumPers,FamNumAdults,FamNumPers1084,FamNumDrivLic,HousehNumPers,HousehNumAdults,HousehNumPers1084,HousehNumDrivLic,DayStartMuncode,DayStartOldMuncode,DayStartCityCode,DayStartNTMzone,DayStartJourneyRole,DayStartPurp,RespNotripReason,NightsAway,TotalNumTrips,NumTripsCorr,NumTripsExclComTrans,TotalLen,TotalLenExclComTrans,TotalMotorLen,TotalBicLen,TotalMin,TotalMotorMin,PrimModeDay,ModeChainTypeDay,DayNumJourneys,JstartType,JstartMuncode,JstartNTMzone,JstartDistNearestStation,DayJourneyType,DayPrimTargetMuncode,DayPrimTargetPurp,SessionWeight
count,146072.000000,146072.000000,146072.000000,146072.000000,146072.000000,146072.000000,146072.000000,146072.000000,142285.000000,146071.000000,1.208840e+05,146066.000000,144475.000000,35789.000000,146072.000000,146072.000000,146072.000000,146072.000000,145924.000000,146051.000000,103376.000000,100966.000000,101911.000000,73893.000000,48804.000000,66830.000000,68000.000000,104106.000000,101279.000000,104085.000000,101085.000000,30066.000000,98378.000000,17706.000000,88252.000000,146071.000000,146072.000000,16116.000000,146068.000000,113630.000000,110329.000000,146054.000000,146048.000000,146071.000000,145929.000000,145926.000000,108535.000000,107953.000000,63692.000000,63358.000000,96449.000000,95945.000000,96358.000000,95857.000000,96188.000000,95688.000000,145915.000000,145915.000000,145915.000000,145915.000000,145915.000000,145915.000000,145915.000000,145915.000000,145915.000000,145915.000000,146066.000000,145451.000000,145451.000000,145451.000000,146045.000000,140060.000000,143707.000000,145832.000000,132525.000000,145726.000000,24110.000000,4090.000000,146072.000000,146072.000000,146072.000000,146072.000000,145770.000000,146072.000000,146072.000000,146072.000000,146072.000000,121927.000000,121864.000000,146072.000000,132138.000000,132034.000000,132103.000000,130324.000000,146072.000000,145260.000000,145899.000000,146072.000000
mean,212707.437284,1.648304,15042.796313,2010.689496,6.491080,3.937538,16.568562,479.903739,471.440538,7864.381773,2.940475e+05,480187.616447,4.257505,46.815809,1.515431,1966.062182,44.627314,44.139383,19.565349,8.660543,471.207756,463.229285,470894.298084,37.231846,1.941070,1.439234,0.684529,476.760869,467.870062,477022.583504,13.388129,12.882570,4.701244,-20.345194,12.071942,1.284855,1.844857,1.692480,-1.436728,1981.897791,1.996420,1.185870,1.073969,1.940871,1.924244,1.347224,262.480453,212.932897,311.321312,252.688469,558.190577,453.387274,563.917547,458.125677,568.878332,462.071827,17.963410,12.685968,2.679300,1.923051,2.384354,1.690964,2.738704,1.971997,2.440537,1.729459,2.780366,2.000536,2.474228,1.751758,489.459735,473.703149,7859.933218,488303.260306,0.055499,4.097196,75.473496,515.016381,2.939448,3.115238,2.931753,39.407442,38.001972,37.141018,1.447076,55.983186,39.371988,10.760693,23.817132,1.200435,2.164971,480.009732,480281.631666,4.263406,11.031019,488.524191,23.751643,365.570729
std,92460.984650,1.213426,1060.348038,2.923696,3.476709,2.047853,8.288357,249.980396,248.195728,6203.745656,4.825574e+05,249857.713341,5.536378,73.300590,0.499764,20.157295,19.976190,19.981995,13.661819,4.68

In [ ]:
print(ses_df.dtypes.to_string())

In [13]:
# Defining the 'categorical' and 'numerical' lists and changing the datatypes accordingly.

# PC
# categorical = ['RespSex', 'RespEdulevel', 'RespPrimOcc', 'HomeAdrMunCode', 'PrimOccMuncode', 'HousehAccomodation']

# Server. Think whether  'WorkHourType' and 'HousehAccomodation' variables are important or not
categorical = ['RespSex', 'RespEdulevel', 'RespMainOccup', 'HomeAdrMunCode', 'PrimOccMuncode']
numerical = ['RespAgeCorrect', 'IncRespondent', 'SessionWeight', 'DiaryYear', 'TotalMin']

all_vars = categorical + numerical

#ses_df = ses_df[all_vars]
ses_df[categorical] = ses_df[categorical].astype('category')

In [14]:
# Decribe categorical data including a "isnull" row
ses_df[categorical].describe().append(ses_df[categorical].isnull().sum().rename('isnull'))

,RespSex,RespEdulevel,RespMainOccup,HomeAdrMunCode,PrimOccMuncode
count,146072,146051.0,145924.0,146072,103376.0
unique,2,11.0,13.0,99,101.0
top,2,11.0,30.0,101,101.0
freq,75290,32991.0,66848.0,11408,11156.0
isnull,0,21.0,148.0,0,42696.0


In [15]:
# Decribe numerical data including a "isnull" row
ses_df[numerical].describe().append(ses_df[numerical].isnull().sum().rename('isnull'))

,RespAgeCorrect,IncRespondent,SessionWeight,DiaryYear,TotalMin
count,146072.000000,108535.000000,146072.000000,146072.000000,146072.000000
mean,44.139383,262.480453,365.570729,2010.689496,55.983186
std,19.981995,230.327332,255.304890,2.923696,61.180042
min,6.000000,0.000000,0.000000,2006.000000,0.000000
25%,28.000000,120.000000,183.597674,2009.000000,14.000000
50%,45.000000,250.000000,308.547179,2010.000000,40.000000
75%,60.000000,360.000000,478.942810,2013.000000,80.000000
max,102.000000,11000.000000,4169.091957,2017.000000,1375.000000
isnull,0.000000,37537.000000,0.000000,0.000000,0.000000


In [16]:
# What are the values on home and occupation codes on the TU data
occ_cat = set(list(ses_df.PrimOccMuncode.cat.categories))
hom_cat = set(list(ses_df.HomeAdrMunCode.cat.categories))

In [ ]:
occ_cat ^ hom_cat

In [18]:
# Which of these appear on the names_codes dataframe we created earlier?
diff = list(set(names_codes.Work_c) ^ occ_cat)
diff = [int(i) for i in diff]
diff

[1, 2, 3, 4, 5, 6, 7, 8, 9, 11, 10, 81, 82, 83, 84, 85, 950, 997, 999]

According to the TU documentation, 997 is the "continental shell" and 999 is "abroad". That makes sense with the lack of 950 on the TU base which in the DST bases is "Uden for Danmark"

In [ ]:
names_codes.loc[names_codes['Work_c'].isin(diff)]

Before merging the data and working out the methodology, we pre process the data by deleting the municipality codes which are not in all databases and the municipality codes for outside of Denmark. We are also going to aggregate the data based on the characteristics that appear on the merging databases

In [19]:
## At first, it seems that we dont need any of the pop and cm matrices we have. We could just aggregate the data on the TU
#   dataset and run the gibbs sampling on it. This needs further discussion.

ses_df = ses_df.drop(ses_df[ses_df.PrimOccMuncode.isin(diff) | # Occupation municipality
                            ses_df.HomeAdrMunCode.isin(diff)].index) # Home municipality

In [ ]:
ses_df[categorical].describe().append(ses_df[categorical].isnull().sum().rename('isnull'))

# Tasks to do:
- Drop NaNs from the database OK
- Re define the categories for all categorical variables so that they match the job, pop and cm databases. OK
- Aggregate the session database by the variables that appear on those bases OK
- Try a first merge and check whether the results are valid or not OK

In [20]:
# Na's from the session database
# Any variable
for var in categorical:
    ses_df.drop(ses_df[ses_df[var].isnull()].index, inplace=True)
# Uknown category from occupation
ses_df.drop(ses_df[ses_df['RespMainOccup']==9.].index, inplace=True)   

print('The length of the ses data is:', len(ses_df))

# Na's from the population database
pop_df.drop(pop_df[pop_df.edu=='H90'].index, inplace=True) # Non-stated education
pop_df.drop(pop_df[pop_df.Sector=='X'].index, inplace=True) # Non-stated activity
pop_df['Sector'] = pop_df.Sector.cat.remove_unused_categories() # Remove unused categories

print('The length of the pop data is:', len(pop_df))

The length of the ses data is: 103195
The length of the pop data is: 22581504


# Categorical variables recoding 
For the job database:
- age: merge (or drop) -15 with with 16-19 and 67+ with 65-66. In pop we have 15-19 and 66-69
- socio: I think in principle we don't care about this variable. This is the type of employment. What we care about (I think) is whether the person is employed or not.

For the pop database:
- PopSocio is a very important variable. It is important that the "employed" category broadly matches the job database figures.
- age: merge the masters and PhD programs into a category called H99 

For the TU data:
- RespAgeCorrect: Get the age into the same categories as in the pop and job databases
- Education: Ask

In [ ]:
#### Job
job_df['AgeGroup'] = np.where(job_df['AgeGroup']=='-15', '15-19', job_df['AgeGroup'])

# Turning it back again to category
job_df['AgeGroup'] = job_df['AgeGroup'].astype('category')

In [21]:
#### Pop
# Education
pop_df['edu'] = pop_df['edu'].astype(str)
pop_df['edu'][(pop_df['edu'] =='H70') | (pop_df['edu'] =='H80')] = 'H99'
pop_df['edu'] = pop_df['edu'].astype('category')

/home/sergio/py3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [22]:
#### TU
# Age
age_bins = [-1, 19, 24, 29, 34, 39, 44, 49, 54, 59, 64, 200]
age_cats = list(pop_df.AgeGroup.cat.categories)

# Create the categorical variable
ses_df['AgeGroup'] = pd.cut(ses_df.RespAgeCorrect, age_bins, labels=age_cats)

# Education, still in process
ses_df['edu'] = ''
ses_df['edu'][(ses_df['RespEdulevel'] ==1.) | 
                    (ses_df['RespEdulevel']==2.) | 
                    (ses_df['RespEdulevel']==3.) | 
                    (ses_df['RespEdulevel']==4.)] = 'H10'
ses_df['edu'][ses_df['RespEdulevel'] ==5.] = 'H20'
ses_df['edu'][ses_df['RespEdulevel'] ==6.] = 'H50'
ses_df['edu'][ses_df['RespEdulevel'] ==9.] = 'H35'
ses_df['edu'][ses_df['RespEdulevel'] ==11.] = 'H30'
ses_df['edu'][ses_df['RespEdulevel'] ==12.] = 'H40'
ses_df['edu'][(ses_df['RespEdulevel'] ==13.) | (ses_df['RespEdulevel'] ==14.)] = 'H99'

# Reset it to be a category
ses_df['edu'] = ses_df['edu'].astype('category')

# Main occupation of the respondent
ses_df['PopSocio'] = ''
ses_df['PopSocio'][(ses_df['RespMainOccup'] ==1.) | 
                    (ses_df['RespMainOccup']==2.) | 
                    (ses_df['RespMainOccup']==3.)] = '0'
ses_df['PopSocio'][(ses_df['RespMainOccup'] ==22.) | 
                    (ses_df['RespMainOccup']==30.) | # ?
                    (ses_df['RespMainOccup']==50.) | # ?
                    (ses_df['RespMainOccup']==52.)] = '1' # ?
ses_df['PopSocio'][(ses_df['RespMainOccup'] ==11.)] = '2' # ?
ses_df['PopSocio'][(ses_df['RespMainOccup'] ==15.) | 
                    (ses_df['RespMainOccup']==20.) | # ?
                    (ses_df['RespMainOccup']==12.) | # ?
                    (ses_df['RespMainOccup']==10.)] = '3' # ?

# Reset it to be a category
ses_df['PopSocio'] = ses_df['PopSocio'].astype('category')

# RespSex, change the names of the categories
ses_df['RespSex'] = ses_df.RespSex.cat.rename_categories({1.: 'M', 2.: 'K'})

/home/sergio/py3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/sergio/py3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/sergio/py3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/sergio/py3/lib/python3.6/site-packages/ipykernel_launcher.py:17: Se

In [ ]:
# Drop the variables that were used to create the new ones
#ses_df = ses_df.drop(['RespAgeCorrect', 'RespMainOccup', 'RespEdulevel'], axis=1)

# Data Aggregation
Pop database
- We have to aggregate the database by the last education change.

TU
- We aggregate the database by all the variables on the pop and job data. 

In [23]:
# Pop database
pop_agg = ['MunicipalityOrigin', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year'] # Variables we group by

pop_df = pop_df.rename(index=str, columns={'Municipality':'MunicipalityOrigin'})
pop_df = pop_df.groupby(pop_agg + ['Sector'], as_index=False).sum()
len(pop_df)

20072448

In [24]:
# Change the names of the sesssion database in order to make an easier merge
ses_names = {
    'PrimOccMuncode':'MunicipalityDest',
    'HomeAdrMunCode':'MunicipalityOrigin',
    'RespSex':'Gender',
    'DiaryYear':'Year',
}
ses_df = ses_df.rename(index=str, columns=ses_names)

# Merge
### First merge strategy
The steps we have to follow to finish the second merge strategy are the following:
- Sample industries from the population database for certain combinations of socio-economic variables
- Input those industries to the individuals according to their socio-economics variables.

### Second merge strategy
The steps we have to follow to finish the second merge strategy are the following:
- Estimate the total population in a certain combination of the pop database.
- Estimate the percentage of population working in each sector for each combination in the pop database.
- Create variables for each sector for each combination
- Merge using the origin municipality
- Multiply the percentages by the aggregated population weights

In [25]:
# We get the combinations totals
pop_df['sums'] = pop_df.groupby(pop_agg, as_index=False)['Val'].transform(sum)

In [26]:
# Correct length = 99 (mun) * 8 (edu) * 4 (PopSocio) * 11 (AgeGroup) * 2 (Gender) * 8 (Year) = 557568
print('Database length is:', len(pop_df)) 

# How many combinations have 0 occurrences ?
print('The number of zeros in the final database is:', sum(pop_df.sums==0)) # 371925, more than  half of the combinations

# How many combinations have 0 occurences in the pop database ? 
print('The number of zeros in the pop database is (Val):', sum(pop_df.Val==0)) 
## 17814693, this is a massive number, this divided by the number of industrial sectors (37) is 481478 which is less than 
#  the number of zero values in sums

Database length is: 20072448
The number of zeros in the final database is: 13389300
The number of zeros in the pop database is (Val): 17814693


In [27]:
# Replace the zero valued sums with 1s
pop_df['sums'][(pop_df['sums']==0.)] = 1

# We divide the values by the totals to get the percentages
pop_df['percent'] = pop_df.Val/pop_df.sums
pop_df = pop_df.drop(['Val', 'sums'], axis=1)

/home/sergio/py3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


First Merging strategy

In [28]:
# Make copies of the databases and make the indices of the databases the common variables.
idx_list = ['MunicipalityOrigin', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year']

# We convert the data types of the variables that don't have the same type, otherwise, merging won't be possible.
for var in ['MunicipalityOrigin', 'edu', 'PopSocio', 'AgeGroup', 'Gender']: 
    pop_df[var] = pop_df[var].astype(str)
    pop_df[var] = pop_df[var].astype('category')
    ses_df[var] = ses_df[var].astype(str)
    ses_df[var] = ses_df[var].astype('category')
    
ses_df_c = ses_df[ses_df.Year>2008].set_index(idx_list)
pop_df_c = pop_df.copy().set_index(idx_list)

In [29]:
# Merge the databases so we have a distribution of industries for each person on TU
merged = ses_df_c.join(pop_df_c, how='inner')

In [30]:
# Drop unnecesary variables
merged = merged[['SessionId', 'Sector', 'percent']].reset_index(drop=True)

In [31]:
# We define the sampling function. This function is necessary when we want to aggregate the data by index
def groupby_sample(x):
    if all(x.percent==0.): # Just in case all of them are 0's
        return np.random.choice(x.Sector)
    else: 
        return np.random.choice(x.Sector, p=x.percent)

In [32]:
# This is the final dataframe. There are as many observations as the original TU data, but in this particular database, we have industries for each person too.
samp_df = merged.groupby('SessionId', as_index=False).aggregate(groupby_sample)
samp_df = samp_df.drop(['percent'], axis=1)

In [33]:
samp_df = samp_df.merge(ses_df_c.reset_index(), on='SessionId', how='inner')

In [34]:
samp_df.describe().append(samp_df.isnull().sum().rename('isnull')).append(samp_df.dtypes.rename('type'))

,SessionId,Year,InterviewType,DiaryDate,DiaryMonth,DiaryWeekday,DiaryDaytype,HomeAdrOldMuncode,HomeAdrCityCode,HomeAdrCitySize,HomeAdrNTMzone,HomeAdrDistNearestStation,HomeParkPoss,RespYearBorn,RespAgeSimple,RespAgeCorrect,PrimOccOldMuncode,PrimOccNTMzone,WorkHoursPw,WorkHourType,WorkPubPriv,WorkatHomeDayspM,SduMuncode,SduOldMuncode,SduNTMzone,GISdistHW,kmarbud,HwDayspW,HwDaysReason,WorkParkPoss,RespHasBicycle,RespHasSeasonticket,RespHasRejsekort,ResphasDrivlic,RespDrivlicYear,RespIsmemCarshare,HousehNumcars,HousehCarOwnership,Handicap,HousehAccomodation,HousehAccOwnorRent,IncRespondent,IncRespondent2000,IncSpouse,IncSpouse2000,IncNuclFamily,IncNuclFamily2000,IncFamily,IncFamily2000,IncHouseh,IncHouseh2000,...,NuclFamNumDrivLic,FamNumPers,FamNumAdults,FamNumPers1084,FamNumDrivLic,HousehNumPers,HousehNumAdults,HousehNumPers1084,HousehNumDrivLic,DayStartMuncode,DayStartOldMuncode,DayStartCityCode,DayStartNTMzone,DayStartJourneyRole,DayStartPurp,RespNotripReason,NightsAway,TotalNumTrips,NumTripsCorr,NumTripsExclComTrans,TotalLen,TotalLenExclComTrans,TotalMotorLen,TotalBicLen,TotalMin,TotalMotorMin,PrimModeDay,ModeChainTypeDay,DayNumJourneys,JstartType,JstartMuncode,JstartNTMzone,JstartDistNearestStation,DayJourneyType,DayPrimTargetMuncode,DayPrimTargetPurp,SessionWeight,AgeGroup,DayStartNUTS,Gender,HomeAdrNUTS,HomeAdrNearestStation,JstartNearestStation,MunicipalityDest,MunicipalityOrigin,PopSocio,PseudoYear,RespEdulevel,RespMainOccup,Sector,edu
count,74752,74752,74752,74752,74752,74752,74752,72533,74752,62722,74747,74084,22005,74752,74752,74752,73145,74106,52742,46887,47479,48527,74749,72514,74738,73934,5021,70553,12957,63376,74751,74752,8712,74750,56316,56126,74752,74748,74751,74657,74654,56958,56958,31976,31976,48693,48693,48649,48649,48666,48666,...,74746,74746,74746,74746,74746,74750,74699,74699,74699,74738,71239,73382,74630,73337,74636,8966,3190,74752,74752,74752,74752,74537,74752,74752,74752,74752,65783,65782,74752,73098,73038,73077,72314,74752,74476,74713,74752,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,241900,2011.66,1.62636,15395.8,6.48078,3.93763,16.554,463.373,7614.52,322977,472911,4.12593,47.8194,1974.72,36.9367,36.4454,459.497,466796,37.1332,1.93926,1.44053,0.656027,472.754,463.478,473034,13.6703,2.64767,4.69511,-20.1139,12.1482,1.24396,1.8264,1.69295,-2.51431,1988.63,1.99501,1.34937,1.20355,1.98507,1.90969,1.31726,296.508,235.916,359.923,286.54,665.778,530.578,671.579,535.227,678.973,541.137,...,1.82752,3.07495,2.06324,2.69733,1.86807,3.13031,2.10613,2.74889,1.90183,483.674,466.061,7616.34,482412,0.0586198,4.4338,94.4406,433.741,3.11364,3.43595,3.12088,45.2485,43.3634,42.709,1.7026,60.6133,43.4917,11.089,25.2314,1.26331,2.18758,473.549,473806,4.17592,11.8322,481.96,22.3776,368.674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,77814.6,2.24585,1.23212,822.851,3.47863,2.05003,8.28012,250.189,6076.82,504092,251827,5.36907,74.9262,16.3703,16.1933,16.1982,255.904,258271,8.6559,1.03995,0.538267,2.36268,251.886,250.082,251732,24.9604,11.3401,0.93904,19.8296,5.03182,0.42947,0.378768,0.461296,7.46313,12.9978,0.0704555,0.830064,0.772034,0.121272,1.19144,0.566084,257.416,204.46,264.947,207.717,407.772,324.37,399.939,317.876,400.339,318.229,...,0.695834,1.34621,0.725039,1.17454,0.718328,1.34901,0.73088,1.17715,0.7178,257.382,249.11,6476.34,254765,0.234913,11.4516,55.4301,2591.27,2.12314,9.49955,2.2792,72.185,67.1611,72.7208,5.21917,60.7848,55.3232,8.29665,39.3975,0.83113,0.698066,251.55,251382,5.41444,17.6246,263.519,16.1101,265.384,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,141385,2009,0,14245,1,1,11,101,0,200,102121,0,4,1925,6,6,101,102100,0.25,1,1,0,101,101,102121,0,0,0,-35,1,1,1,1,-18,1945,1,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,1,0,1,0,101,101,0,102121,0,1,11,1,0,0,0,0,0,0,0,0,0,1,1,0,1,101,102121,0,1,101,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,171022,2010,2,14716,3,2,11,215,1100,3951,230043,0.8,4,1961,21,21,185,201031,37,1,1,0,230,215,230045,1.5

In [ ]:
samp_df.to_csv('sampling_df.txt', sep=',', index=False)

In [35]:
samp_df = pd.read_csv('sampling_df.txt', sep=',')

In [36]:
# After analayzing the correlation between variables, variables with more than 95% of correlation were dropped from the database. This, hopefully will reduce the dimensionality of the input
drop = ['JstartMuncode', 'JstartNTMzone', 'HomeAdrNTMzone', 'SduMuncode', 'SduOldMuncode', 'HomeAdrOldMuncode', 'DayStartOldMuncode', 'SduNTMzone', 'PrimOccNTMzone', 
'PrimOccOldMuncode', 'IncNuclFamily2000', 'IncFamily2000', 'IncFamily', 'DayStartMuncode', 'DayStartNTMzone', 'IncHouseh2000', 'IncNuclFamily', 'IncHouseh', 
'DayPrimTargetMuncode', 'DiaryDate', 'FamNumPers', 'HousehNumDrivLic', 'HousehNumAdults', 'FamNumPers1084', 'RespAgeSimple', 'HousehNumPers', 'IncRespondent', 
'IncSpouse2000', 'RespYearBorn', 'NuclFamNumPers', 'IncSpouse', 'TotalLen', 'NuclFamNumPers1084', 'RespMainOccup', 'RespEdulevel'] 
samp_df.drop(drop, inplace=True, axis=1)

In [37]:
### Numerical variables define
numerical = ['IncRespondent2000', 'TotalLenExclComTrans', 'TotalMotorLen', 'TotalBicLen', 'TotalMin', 'TotalMotorMin', 'SessionId', 'WorkHoursPw', 'GISdistHW', 'HousehNumPers1084',
            'TotalNumTrips', 'NumTripsCorr', 'NumTripsExclComTrans', 'SessionWeight', 'HomeAdrCitySize', 'HomeAdrDistNearestStation', 'HwDayspW', 'WorkatHomeDayspM', 
             'JstartDistNearestStation', 'NightsAway'] # Numerical variables with missing values 
samp_df[numerical] = samp_df[numerical].astype('float64')

In [38]:
def replace_missing(df, grouping_vars = ['MunicipalityOrigin', 'AgeGroup', 'edu', 'PopSocio']):
    """
    Replaces the missing values of numerical variables on the dataframe by taking the mean (or could be median) of the variables grouped by certain variables.
    It makes a two dataframes: one with the nans which we use to replace the missing values and one with the rest of the variables. 
    """
    # Variables we groupby 
    numerical = ['IncRespondent2000', 'TotalLenExclComTrans', 'TotalMotorLen', 'TotalBicLen', 'TotalMin', 'TotalMotorMin', 'WorkHoursPw', 'GISdistHW', 'HousehNumPers1084',
            'TotalNumTrips', 'NumTripsCorr', 'NumTripsExclComTrans', 'SessionWeight', 'HomeAdrCitySize', 'HomeAdrDistNearestStation', 'HwDayspW', 'WorkatHomeDayspM', 
             'JstartDistNearestStation', 'NightsAway'] # Numerical variables with missing values 
    
    # Divide the dataset
    df_auxiliar = df.drop(numerical, axis=1) # Dataframe which we don't modify
    df_missing  = df[grouping_vars+numerical+['SessionId']] # Dataframe with missings

    # NaN replacement
    f = lambda x: x.fillna(np.random.choice(x)) # Function to fill the na values using a random element 
    df_missing = df_missing.groupby(grouping_vars).transform(f) # Applying the function to the grouping
    
    return df_missing.merge(df_auxiliar, on='SessionId')

In [39]:
samp_df = replace_missing(samp_df)
samp_df = replace_missing(samp_df, grouping_vars = ['MunicipalityOrigin', 'edu', 'PopSocio'])
samp_df = replace_missing(samp_df, grouping_vars = ['MunicipalityOrigin', 'PopSocio'])
samp_df = replace_missing(samp_df, grouping_vars = ['PopSocio'])
samp_df[numerical] = samp_df[numerical].fillna(samp_df[numerical].mean())
samp_df[numerical] = samp_df[numerical].astype('int32')

# Drop AgeGroup since we don't need it anymore
samp_df.drop(['AgeGroup'], inplace=True, axis=1)

In [40]:
samp_df = samp_df.dropna(thresh=0.2*len(samp_df), axis=1) # Drop columns with more than 20% of missing values

In [41]:
### Categorical variables define 
categorical = [col for col in list(samp_df) if col not in numerical]

samp_df[categorical] = samp_df[categorical].astype('category')
# Convert categorical to categorical data type
for cat in categorical:
    samp_df[cat] = samp_df[cat].cat.rename_categories({'nan': 'MISS'})
samp_df[categorical] = samp_df[categorical].astype('str')
samp_df[categorical].replace(to_replace=['NaN', 'nan'], value='MISS') # Replace NaN with MISS as a new category on categorical data so we don't have a problem in keras.
samp_df[categorical] = samp_df[categorical].astype('category')

In [42]:
samp_df[categorical].head()

,Sector,MunicipalityOrigin,edu,PopSocio,Gender,Year,InterviewType,PseudoYear,DiaryMonth,DiaryWeekday,DiaryDaytype,HomeAdrNUTS,HomeAdrCityCode,HomeAdrNearestStation,HomeParkPoss,RespAgeCorrect,MunicipalityDest,WorkHourType,WorkPubPriv,WorkParkPoss,RespHasBicycle,RespHasSeasonticket,ResphasDrivlic,RespDrivlicYear,RespIsmemCarshare,HousehNumcars,HousehCarOwnership,Handicap,HousehAccomodation,HousehAccOwnorRent,NuclFamType,PosInFamily,NuclFamNumAdults,NuclFamNumDrivLic,FamNumAdults,FamNumDrivLic,DayStartNUTS,DayStartCityCode,DayStartJourneyRole,DayStartPurp,PrimModeDay,ModeChainTypeDay,DayNumJourneys,JstartType,JstartNearestStation,DayJourneyType,DayPrimTargetPurp
0,QB,219,H30,1,K,2009,2,2008/9,1,4,32,DK013,10076.0,Skævinge ...,nan,46,270.0,nan,1.0,11.0,1.0,2,1.0,1983.0,2.0,1.0,1.0,2.0,2.0,1.0,21.0,11.0,3.0,2.0,3.0,2.0,DK013,10076.0,0.0,1.0,11.0,11.0,1.5,2.0,Skævinge ...,12,11.0
1,P,259,H50,1,K,2009,2,2008/9,1,4,32,DK021,10048.0,Ølby ...,nan,41,253.0,nan,1.0,11.0,2.0,2,1.0,1995.0,2.0,1.0,1.0,2.0,1.0,1.0,11.0,10.0,1.0,1.0,1.0,1.0,DK022,10307.0,1.0,41.0,11.0,11.0,1.5,2.0,Ølby ...,21,41.0
2,QA,849,H50,1,M,2009,2,2008/9,1,4,32,DK050,11269.0,Lindholm ...,nan,33,849.0,nan,1.0,nan,1.0,2,1.0,1994.0,2.0,1.0,1.0,2.0,1.0,1.0,21.0,12.0,2.0,1.0,2.0,1.0,DK050,10929.0,1.0,41.0,11.0,11.0,0.5,2.0,Lindholm ...,21,41.0
3,O,159,H99,1,K,2009,2,2008/9,1,4,32,DK012,1100.0,Kildebakke ...,nan,45,175.0,nan,2.0,12.0,1.0,2,2.0,nan,nan,0.0,0.0,2.0,3.0,2.0,11.0,10.0,2.0,0.0,2.0,0.0,DK012,1100.0,0.0,1.0,2.0,2.0,1.0,2.0,Kildebakke ...,11,41.0
4,P,779,H99,1,K,2009,2,2008/9,1,4,32,DK041,10791.0,Skive ...,nan,56,791.0,nan,2.0,13.0,1.0,2,1.0,1971.0,2.0,1.0,1.0,2.0,1.0,1.0,20.0,12.0,2.0,2.0,2.0,2.0,DK041,10791.0,0.0,1.0,nan,nan,0.0,3.0,Skive ...,1,1.0


In [43]:
samp_df['PrimModeDay'].cat.categories

Index(['1.0', '11.0', '12.0', '13.0', '14.0', '15.0', '2.0', '25.0', '26.0',
       '3.0', '31.0', '32.0', '33.0', '34.0', '35.0', '4.0', '41.0', '42.0',
       '5.0', '51.0', '6.0', '7.0', 'nan'],
      dtype='object')

In [44]:
samp_df.to_csv('sampling_df_no_nan.txt', sep=',', index=False)

Second merging strategy

In [ ]:
# Session database
# Since we have NaN on the income variable and we don't want to drop that values,
# I am going to input the mean of the conditionals to that particular individual

# Some variables might have to be deleted before this groupby, the notebook has changed since the last time I made this database.

ses_agg = ['MunicipalityOrigin', 'MunicipalityDest', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year']
ses_df = ses_df.groupby(ses_agg + ['IncRespondent', 'TotalMin'], as_index=False).agg({
    'SessionWeight':'sum', 
    'IncRespondent': [np.nanmean],
    'TotalMin': [np.nanmean]
}).reset_index(drop=True)

In [ ]:
ses_df.head()

In [ ]:
# Changing the database from a long format to a wide format
pop_df = pop_df.set_index(pop_agg + ['Sector']).unstack('Sector')

In [ ]:
# Funcation to the get the indices as variables
def reset_index(df): # https://github.com/pandas-dev/pandas/issues/19136
  '''Returns DataFrame with index as columns'''
  index_df = df.index.to_frame(index=False)
  df = df.reset_index(drop=True)
  #  In merge is important the order in which you pass the dataframes
  # if the index contains a Categorical. 
  # pd.merge(df, index_df, left_index=True, right_index=True) does not work
  return pd.merge(index_df, df, left_index=True, right_index=True)

In [ ]:
pop_df = reset_index(pop_df)

In [ ]:
pop_df.dtypes.head(10)

In [ ]:
ses_df[['MunicipalityOrigin', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year']].dtypes

In [ ]:
test_list = ['MunicipalityOrigin', 'edu', 'PopSocio', 'AgeGroup', 'Gender']
for var in test_list: 
    print(var, 'pop:', pop_df[var].cat.categories.dtype, 'ses:', ses_df[var].cat.categories.dtype)

In [ ]:
# As it is possible to see, the levels of the categories in both datasets do not match. Therefore we have to convert them in 
# order to keep working with them.
for var in test_list: 
    pop_df[var] = pop_df[var].astype(str)
    pop_df[var] = pop_df[var].astype('category')
    ses_df[var] = ses_df[var].astype(str)
    ses_df[var] = ses_df[var].astype('category')
    print(var, 'pop:', pop_df[var].cat.categories.dtype, 'ses:', ses_df[var].cat.categories.dtype)

In [ ]:
# Finally merging both datasets
com_df = pd.merge(pop_df, ses_df, indicator=True, validate='1:m')
com_df.head()

In [ ]:
# Merge check
print(com_df._merge.describe())
com_df = com_df.drop(['_merge'], axis=1)

In [ ]:
# Change the names of the dataframe to work more easily with them
names = list(com_df)
for idx in range(6, 42):
    names[idx] = names[idx][1] 

# Renaming
com_df.columns = names
# Checking
list(com_df)

In [ ]:
# Changing the format from wide to long
com_df = pd.melt(com_df, var_name='Sector', id_vars=['MunicipalityOrigin', 'MunicipalityDest', 'edu', 
                                                     'PopSocio', 'AgeGroup', 'Gender', 'Year', 'SessionWeight', 
                                                     'IncRespondent', 'TotalMin'])

In [ ]:
# Rounding the value counts so we get integers
com_df['counts'] = round((com_df.SessionWeight * com_df.value) + 0.5).astype('int')
# Dropping unnecesary variables and converting the municipality type
com_df = com_df.drop(['value', 'SessionWeight'], axis=1)
com_df['MunicipalityOrigin'] = com_df['MunicipalityOrigin'].astype('int')
com_df['MunicipalityDest'] = com_df['MunicipalityDest'].astype('int')

In [ ]:
com_df

In [ ]:
com_df.to_csv('/mnt/sdb1/data_sergio/com_df.txt', sep=',')

# Locations and distances

Out of personal curiosity I would like to build the distance matrix between the municipality of origin and destination. Using the google API and the 'googlemaps' module on python, we can retrieve the exact latitude (lat) and longitude (lon) of the names and the distance (in different transport modes) between to points. The steps that I would have to follow in order to get the information are the following:

- From the names_codes dataframe, eliminate whatever that is not present in the com_df database in order to optimize the API's calls.

- Merge the names_codes database with the lat and lon from the previous step with the com_df database (make sure to get both the lat and lon from the origin and destination.

- Aggregate the data by the origin and destination so that we don't get repeated calls in the API. Get the distances for as many modes of transportation as possible.

In [ ]:
from geopy.geocoders import Nominatim # Module to retrieve the geolocations of the municipalities DONT USE IF NOT NEEDED
import simplejson
import urllib

In [ ]:
names_codes = names_codes.drop(names_codes[names_codes['Work_c'].isin(diff)].index)

In [ ]:
names_codes

In [ ]:
### UNCOMMENT ONLY IF YOU DONT HAVE ACCESS TO THE LatLon_Mun.txt DATABASE

#geolocator = Nominatim()

#names_codes['lat'] = 0.
#names_codes['lon'] = 0. 

#for index, row in names_codes.iterrows():
#    loc = geolocator.geocode(str(row['Work_v'] + ', ' + 'Danmark'))
#    names_codes['lat'][index] = loc.latitude
#    names_codes['lon'][index] = loc.longitude

# names_codes['Work_c'] = names_codes['Work_c'].astype('int')
# names_codes.to_csv('/mnt/sdb1/data_sergio/LatLon_Mun.txt', sep=',')

In [ ]:
# Extract the combinations of orgin and destination present in the created dataset
combinations = com_df[['MunicipalityOrigin', 'MunicipalityDest']].groupby(['MunicipalityOrigin', 'MunicipalityDest'], 
                                                                          as_index=False).sum()
# Merge latlon for origin
# Merging the combinations database with the latitude and longitude
combinations = pd.merge(combinations, names_codes, left_on='MunicipalityOrigin', right_on='Work_c', validate='m:1')
combinations = combinations.rename(index=str, columns={'lat':'orig_lat', 'lon':'orig_lon'})

# Merge latlon for destination
combinations = pd.merge(combinations, names_codes, left_on='MunicipalityDest', right_on='Work_c', validate='m:1')
combinations = combinations.rename(index=str, columns={'lat':'dest_lat', 'lon':'dest_lon'})

# Leave the relevant information only
combinations = combinations[['MunicipalityOrigin', 'MunicipalityDest', 'orig_lat', 'orig_lon', 'dest_lat', 'dest_lon']]

# Create the duration column
combinations['drive_time'] = 0.
combinations['cycling_time'] = 0.

# Create the distance column
combinations['drive_dist'] = 0.
combinations['cycling_dist'] = 0.

In [ ]:
for index, row in combinations.iterrows():
    if int(index) < 8671:
        continue
    
    orig_coord = str(row.orig_lat) + ',' + str(row.orig_lon)
    dest_coord = str(row.dest_lat) + ',' + str(row.dest_lon)

    # Driving
    try:
        url = "http://maps.googleapis.com/maps/api/distancematrix/json?origins={0}&destinations={1}&mode=driving&language=en-EN&sensor=false".format(str(orig_coord),str(dest_coord))
        result = simplejson.load(urllib.request.urlopen(url))
        combinations['drive_dist'][index] = result['rows'][0]['elements'][0]['distance']['value']
        combinations['drive_time'][index] = result['rows'][0]['elements'][0]['duration']['value']
    except:
        combinations['drive_dist'][index] = -99
        combinations['drive_time'][index] = -99
        
    # Bycicling
    try:
        url = "http://maps.googleapis.com/maps/api/distancematrix/json?origins={0}&destinations={1}&mode=bicycling&language=en-EN&sensor=false".format(str(orig_coord),str(dest_coord))
        result = simplejson.load(urllib.request.urlopen(url))
        combinations['cycling_dist'][index] = result['rows'][0]['elements'][0]['distance']['value']
        combinations['cycling_time'][index] = result['rows'][0]['elements'][0]['duration']['value']
    except:
        combinations['cycling_dist'][index] = -99
        combinations['cycling_time'][index] = -99

    # public transport
    #url = "http://maps.googleapis.com/maps/api/distancematrix/json?origins={0}&destinations={1}&mode=transit&language=en-EN&sensor=false".format(str(orig_coord),str(dest_coord))
    #result = simplejson.load(urllib.request.urlopen(url))
    #public_distance = result['rows'][0]['elements'][0]['distance']['value']
    #public_time = result['rows'][0]['elements'][0]['duration']['value']

In [ ]:
# Save the dataset for future use:
combinations.to_csv('/mnt/sdb1/data_sergio/Combinations_Mun.txt', sep=',')
# Se quedo en 8671

# Load the dataset
# combinations = pd.read_csv('Combinations_Mun.text', sep=',')

In [ ]:
combinations[8650:8800]

In [ ]:
result

In [ ]:
# https://console.developers.google.com/apis/credentials?project=_ API key for public transportation distance

In [ ]:
df = pd.read_csv('TU0616_csv/session.csv', sep=',')

In [ ]:
df['IncRespondent'][df.IncRespondent==0] = np.nan

In [ ]:
df['IncRespondent'] = df.IncRespondent-df.IncRespondent.mean()
df['IncRespondent'] = df.IncRespondent/df.IncRespondent.var()

In [ ]:
df['IncRespondent'].hist()

In [ ]:
df['RespAgeCorrect'].hist(bins=30)

In [ ]:
df['TotalMin'].hist(bins=50)

We would like to clean the data to make it easier to work with on the VAE.

In [ ]:
samp_df = pd.read_csv('sampling_df_no_nan.txt', sep=',', index_col=0)

In [ ]:
corr_matrix = samp_df.corr().abs().unstack().sort_values(kind="quicksort", ascending=False)

In [ ]:
mask = (corr_matrix > 0.95) | (corr_matrix == np.nan)

In [ ]:
mask = mask[mask].reset_index()
# 'JstartMuncode', 'JstartNTMzone', 'HomeAdrNTMzone', 'SduMuncode', 'SduOldMuncode', 'HomeAdrOldMuncode', 'DayStartOldMuncode', 'SduNTMzone', 'PrimOccNTMzone', 
# 'PrimOccOldMuncode', 'IncNuclFamily2000', 'IncFamily2000', 'IncFamily', 'DayStartMuncode', 'DayStartNTMzone', 'IncHouseh2000', 'IncNuclFamily', 'IncHouseh', 
# 'DayPrimTargetMuncode', 'DiaryDate', 'FamNumPers', 'HousehNumPers1084', 'HousehNumDrivLic', 'HousehNumAdults', 'FamNumPers1084', 'WorkHoursPw', 'RespAgeSimple', 
# 'HousehNumPers', 'TotalMotorLen', 'SessionId', 'IncRespondent', 'IncSpouse2000', 'RespYearBorn', 'NuclFamNumPers', 'IncSpouse', 'WorkatHomeDayspM', 'TotalLen', 
# 'NuclFamNumPers1084', 'RespMainOccup', 'RespEdulevel', 'AgeGroup'
#corr_matrix.reset_index()[corr_matrix.reset_index()['level_0']=='TotalLen'] # This function helps to check the correlation matrix for a specific variable

# Next steps

The pending work is the following.
- Generate apropiate weights for the population data. This can be done using the ipf algorithm. (Check if it is already implemented in python).
- Create the gibbs sampler for the data we currently have. OK. Ask what we should do about being stuck in local spaces.
- Make normality tests for the income data we have. OK. Not a single combination gives normality for all the TU data, hence, we cannot sample from a parametric distribution on the sampling fase.
- Implement and use the VAE and GAN for the data we have. VAE is already implented, this is just a matter of trying it out. GANs, look for a basic implementation and tweak it so that it works with our data. 

In [ ]:
from sklearn.preprocessing import Normalizer

In [ ]:
gibbs_list = ['MunicipalityOrigin', 'MunicipalityDest', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year', 'Sector']
gibbs = com_df[gibbs_list].sample() # Sample initialization

In [ ]:
gibbs

In [ ]:
gibbs_list = ['MunicipalityOrigin', 'MunicipalityDest', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year', 'Sector']
gibbs = com_df[gibbs_list].sample() # Sample initialization
n_samples = 1

#### Gibbs sampler
for n in range(1, n_samples):
    gibbs = gibbs.append(gibbs.iloc[[n-1]]).reset_index(drop=True)
    for s_v in gibbs_list:
        temp_vars = list(set(gibbs_list) - set([s_v])) # List of fixed variables  
        i1 = com_df.set_index(temp_vars).index # Make the index of the complete dataset the temporal vars
        i2 = gibbs.set_index(temp_vars).index # Make the index of the gibbs samples the temporal vars
        
        dist = com_df[i1.isin(i2)].counts/sum(com_df[i1.isin(i2)].counts) # Get the distribution on the counts
        if all(dist==0.): # Just in case all of them are 0's
            idx = np.random.choice(dist.index)
        else: 
            idx = np.random.choice(dist.index, p=dist) #
        gibbs.loc[n, s_v] = com_df[gibbs_list].loc[idx, s_v] # Replace the row with the new value

In [ ]:
gibbs

In [ ]:
hmm_list = ['MunicipalityOrigin', 'MunicipalityDest', 'edu', 'PopSocio', 'AgeGroup', 'Gender', 'Year', 'Sector']
hmm = com_df[hmm_list].sample() # Sample initialization
n_samples = 50

#### Fake HMM sampler
for n in range(1, n_samples):
    hmm = hmm.append(hmm.iloc[[n-1]]).reset_index(drop=True)
    for s_v in hmm_list:
        i1 = com_df.set_index(s_v).index # Make the index of the complete dataset the temporal vars
        i2 = hmm.set_index(s_v).index # Make the index of the gibbs samples the temporal vars
        
        dist = com_df[i1.isin(i2)].counts/sum(com_df[i1.isin(i2)].counts) # Get the distribution on the counts
        if all(dist==0.): # Just in case all of them are 0's
            idx = np.random.choice(dist.index)
        else: 
            idx = np.random.choice(dist.index, p=dist) #
        hmm.loc[n, s_v] = com_df[hmm_list].loc[idx, s_v] # Replace the row with the new value